### Mathematics

In the following, we'll work with the mixed form the of the diffusion equation.
The fields we're solving for are the scalar concentration field $\phi$ and the flux $u$, a vector field.
The inputs to the problem are the initial concentration $\phi|_{t = 0}$, the conductivity $k$, and the sum $f$ of sources and sinks.
The variational form of the equation is

$$\int_\Omega\left\{\left(\partial_t\phi + \nabla\cdot u\right)\psi + \phi\nabla\cdot v - k^{-1}u\cdot v\right\}\;dx = \int_\Omega f\cdot \psi\; dx$$

for all test functions $\psi$, $v$.
We will impose one additional constraint: **the concentration is always positive**, i.e. $\phi \ge 0$ throughout the entire domain.
This makes the problem into a variational inequality or complementarity problem.

We'll want to find an exactly solvable instance of this problem in order to check the order of convergence of our numerical solver.
The right-hand side will be radially symmetric, with sources near the origin and sinks away from the origin:
$$f = \begin{cases}f_1 (1 - (r / r_1)^2)^2 & r \le r_1 \\ 0 & r_1 < r \le r_2 \\ -f_2(1 - r/r_2)^2 & r_2 < r\end{cases}$$
Where the 0-contour falls will depend on the radii and amplitudes of the sources and sinks as well as the conductivity of the medium.
The exact solution will satisfy

$$\phi(r) = c - \int_0^r\frac{1}{r'}\int_0^{r'}k^{-1} r''f(r'')dr''\;dr'$$

In [ ]:
import sympy

r, s, t, r_1, r_2, f_1, f_2 = sympy.symbols("r s t r_1 r_2 f_1 f_2", real=True, positive=True)
f = sympy.Piecewise(
    (f_1 * (1 - (t / r_1)**2)**2, t <= r_1),
    (0, (r_1 < t) & (t <= r_2)),
    (-f_2 * (1 - t / r_2)**2, r_2 < t)
)

In [ ]:
g = sympy.integrate(t * f, [t, 0, s])

In [ ]:
ϕ = sympy.integrate(g / s, [s, 0, r])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Φ = sympy.lambdify(r, ϕ.subs([(r_1, 1.0), (r_2, 2.0), (f_1, 1.0), (f_2, 1.0)]), modules="numpy")

In [ ]:
rs = np.linspace(0.01, 4.0, 101)
Φs = np.array([Φ(r) for r in rs])

In [ ]:
fig, ax = plt.subplots()
ax.plot(rs, Φs);

### Numerics

Now we'll actually try implementing this using Firedrake.

In [ ]:
from numpy import pi as π
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import tqdm
import firedrake
from firedrake import inner, grad, div, dx, Constant, as_vector
import irksome
from irksome import Dt

In [ ]:
mesh = firedrake.UnitDiskMesh(4)
cg = firedrake.FiniteElement("Bernstein", "triangle", 2)
#cg = firedrake.FiniteElement("CG", "triangle", 2)
b = firedrake.FiniteElement("Bubble", "triangle", 4)
Q = firedrake.FunctionSpace(mesh, cg)
V = firedrake.VectorFunctionSpace(mesh, cg + b)
Z = Q * V

In [ ]:
x = firedrake.SpatialCoordinate(mesh)
W = firedrake.FunctionSpace(mesh, "CG", 2)
r = inner(x, x) ** 0.5
r_1 = Constant(0.25)
a_1 = Constant(1.0)
expr = a_1 * firedrake.conditional(r <= r_1, (1 - (r / r_1)**2)**2, 0)
f_1 = firedrake.interpolate(expr, W)

r_2 = Constant(0.75)
a_2 = Constant(2.0)
expr = a_2 * firedrake.max_value(0, (r / r_2 - 1))**2
f_2 = firedrake.interpolate(expr, W)

f = firedrake.interpolate(f_1 - f_2, W)

In [ ]:
fmax = max(float(a_1), float(a_2))
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(f, vmin=-fmax, vmax=fmax, cmap="RdBu_r", axes=ax)
fig.colorbar(colors);

In [ ]:
firedrake.assemble(f * dx)

In [ ]:
z = firedrake.Function(Z)
ϕ, u = firedrake.split(z)
w = firedrake.TestFunction(Z)
ψ, v = firedrake.split(w)

k = Constant(10.0)
F_1 = (Dt(ϕ) + div(u) - f) * ψ * dx
F_2 = -(inner(u, v) / k - ϕ * div(v)) * dx
F = F_1 + F_2

In [ ]:
t = Constant(0.0)
dt = Constant(0.01)
final_time = 5.0
num_steps = int(final_time / float(dt))

In [ ]:
import sia
tableau = irksome.RadauIIA(2)
bcs = None
soln_stages, form = sia.embed(F, z, t, dt, tableau, bcs)

In [ ]:
problem = firedrake.NonlinearVariationalProblem(form, soln_stages)

params = {
    "solver_parameters": {
        "mat_type": "aij",
        "snes_atol": 1e-16,
        "snes_type": "vinewtonrsls",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
}
solver = firedrake.NonlinearVariationalSolver(problem, **params)

In [ ]:
from firedrake.petsc import PETSc

upper = firedrake.Function(soln_stages.function_space())
with upper.dat.vec as upper_vec:
    upper_vec.set(PETSc.INFINITY)

lower = firedrake.Function(soln_stages.function_space())
with lower.dat.vec as lower_vec:
    lower_vec.set(PETSc.NINFINITY)

with lower.sub(0).dat.vec as lower_vec:
    lower_vec.set(0.0)

In [ ]:
ϕs = [firedrake.Function(Q)]

for step in tqdm.trange(num_steps):
    solver.solve(bounds=(lower, upper))
    ϕ = soln_stages.subfunctions[0]
    ϕs.append(ϕ.copy(deepcopy=True))

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(ϕs[-1], axes=ax)
fig.colorbar(colors);

In [ ]:
import numpy as np
xs = np.linspace(0.0, 1.0, 201)
ys = np.zeros_like(xs)
X = np.column_stack((xs, ys))
samples = ϕs[-1].at(X)

In [ ]:
fig, ax = plt.subplots()
ax.plot(xs, samples);